In [1]:
import numpy as np
import torch
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from model_PCA_correlation import AttentionModel_PCA
from dcascore import *
from utils import read_fasta_alignment, remove_duplicate_sequences, add_PCA_coords

# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from model import AttentionModel

from plm_gen_methods import generate_plm_n_save
from seq_utils import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq 


In [2]:
print(current_dir)
print(parent_dir)

c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src


In [3]:
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]


In [4]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 65, 65])
34
65


In [5]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))

##############################################################


In [3]:
"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp)


Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.
[17  5]


In [7]:
"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"

    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b)
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b)

100%|██████████| 15000/15000 [04:39<00:00, 53.71it/s]


Generated sequences (letters): ['W--KYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ--------', 'W--KYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLISVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLLSVWHPDRFGGQRVTEDFLQRINVANSTLKQ---T----', 'W--PYLGLTGPV-LAELKLSWRRLLSVWHPDRFGGQRVTEDFLQRINVAPSTLKQ---T----']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:37<00:00, 53.97it/s]


Generated sequences (letters): ['PEAYFSAYGNRSMTARQVSVNSLYRCMWAWQWVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PEAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIHFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMEWHSGAIEWVDMVQ-AAHIKFHER', 'PPAYFSAYGNRSMTARQVSVNSLYRCMWAWQIVDDTTCMIWHSGAIEWVDMVQ-AAHIKFHER']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:37<00:00, 54.04it/s]


...

Generated sequences (letters): ['DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLSDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLRDQNRRT---', 'DPFKVLQLPPTATPNEITSAYRALVRSFHPDAQERHDRADAQLQEINIAYERLRDQNRRT---']
Generated sequences saved to generated_sequencesPCA_comp


100%|██████████| 15000/15000 [04:38<00:00, 53.94it/s]


Generated sequences (letters): ['DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEISEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD', 'DYYEILGVSKDASEEEIKKAYRKLARKYHPDLNPDDPKAEEKFKEINEAYEVLSDEEKRKQYD']
Generated sequences saved to generated_sequencesPCA_comp


  0%|          | 12/15000 [00:00<04:41, 53.16it/s]c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:52: RuntimeWarning: overflow encountered in exp
  prob = np.exp(self.beta * sum_energy)  # unnormalized
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:63: RuntimeWarning: invalid value encountered in divide
  probs /= probs.sum()
  0%|          | 16/15000 [00:00<05:00, 49.90it/s]


ValueError: probabilities contain NaN

In [2]:
H = 64
d= 10
N = 174
n_epochs = 500
nb_PCA_comp=2
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_youss/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]

In [3]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 63, 63])
21
63


In [4]:
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_2models_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N1=Q_1.shape
_,_,N2=K_1.shape
_,q1,q2=V_1.shape
model=AttentionModel_PCA(H,d,N1,N2,q1,q2,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

# i_indices = torch.arange(L, device=device).unsqueeze(1)
# j_indices = torch.arange(L, device=device).unsqueeze(0)
# mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
# W = W * mask
    
# Compute Jtens
Jtens_PCA = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)
print(Jtens_PCA.shape)

torch.Size([64, 63, 2])
21
63
torch.Size([21, 34, 63, 2])


In [5]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences_2Model_PCA"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]),J_PCA=Jtens_PCA)

##############################################################

"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
print(init_sequence_num[-2:])
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp,J_PCA=Jtens_PCA)

"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"

    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num,nb_PCA_comp=nb_PCA_comp, beta=b,J_PCA=Jtens_PCA)
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b)

100%|██████████| 30000/30000 [09:58<00:00, 50.16it/s]


Generated sequences (letters): ['PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYRIACAAM-ENWDAAFGHK-VSFLMCC', 'PSAEEILYWFHPCHIRVGQKL-YTYGSSAWEVYYYLIACAAM-ENWDAAFGHK-VSFLMCC', 'PSAEEILYWFHDCHIRVGQKL-YTYGSSAWEVYYYLIACAAM-ENWDAAFGHK-VSFLMCC', 'PSAEEILYWFHDCHIRVGQKL-YTYGSTAWEVYYYLIACAAM-ENWDAAFGHK-VSFLMCC', 'PSAEEILYWFHDCHIRVGQKL-YTYGSTAWEVYYYEIACAAM-ENWDAAFGHK-VSFLMCC']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp
Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.
[17  5]


100%|██████████| 30000/30000 [09:40<00:00, 51.72it/s]


Generated sequences (letters): ['NPYEELGEFEGAWQDEIKKVHRE-RKKYHPDQYGDNRFAETKDGELWQAWDKLYVPS---CI-', 'NPYEELGEFEGAWQDEIKKVHRE-RKKYHPDQYGDNRFAETKDGELWQAWDKLYVPS---CI-', 'NPYEELGEFEGAWQDEIKKVHRE-RKKYHPDQYGDNRFAETKDGELWQAWDKLYVPS---CI-', 'NPYEELGEFEGAWQDEIKKRHRE-RKKYHPDQYGDNRFAETKDGELWQAWDKLYVPS---CI-', 'NPYEELGEFEGAWQDEIKKRHRE-RKKYHPDQYGDNRFAETKDGELWQAWDKLYVPS---CI-']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [04:50<00:00, 51.66it/s]


...

Generated sequences (letters): ['EQSH-DRRRAAGAAVSRELAAGEDRVIAGLPAAANLYEDYPAQAA--AQALSHSIYYELMCI-', 'EQSH-DRRRAAGAAVSRELAAGEDRVIAGLPAAANLYEDYPAQAA--ADALSHSIYYELMCI-', 'EQSH-DRRRAAGAAVSRELAAGEDRVIAGLPAAANLYEDYPAQAA--ADALSHSIYYELMCI-', 'EQSH-DRRRAAGAAVSRELAAGEDRVIAGLPAAANLYEDYPAQAA--ADALSHSIYYELMCI-', 'EPSH-DRRRAAGAAVSRELAAGEDRVIAGLPAAANLYEDYPAQAA--ADALSHSIYYELMCI-']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


100%|██████████| 15000/15000 [04:49<00:00, 51.79it/s]


Generated sequences (letters): ['--------------AELAARYRALARQLHPDAAAAARAAARRAARLNAAYALLRDPLRRAAI-', '--------------AELAARYRALARQLHPDAAAAARAAARRAARLNAAYALLRDPLRRAAI-', '--------------AELAARYRALARQLHPDAAAAARAAARRAARLNAAYALLRDPLRRAAI-', '--------------AELAARYRALARQLHPDAAAAARAAARRAARLNAAYALLRDPLRRAAI-', '--------------AELAARYRALARQLHPDAAAAARAAARRAARLNAAYALLRDPLRRAAI-']
Generated sequences saved to generated_sequences_2Model_PCAPCA_comp


  0%|          | 0/15000 [00:00<?, ?it/s]c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:59: RuntimeWarning: overflow encountered in exp
  prob = np.exp(self.beta * sum_energy)  # unnormalized
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:70: RuntimeWarning: invalid value encountered in divide
  probs /= probs.sum()
  0%|          | 3/15000 [00:00<06:46, 36.88it/s]


ValueError: probabilities contain NaN